# LaVague

## Choosing an inference engine

LaVague works with two kind of inference:
- Local
- API with Hugging Face Inference API

Local model allows full control over the experience, but might be slower to setup.
Starting with Hugging Face Inference API is good for a quick start but lacks flexibility and control.

Both options work, but in this Colab notebook, it might take longer to run with local model as weights download can take a while.

### Hugging Face Inference API

To have a fast and low-cost experience, we will use [Hugging Face Inference for PRO users](https://huggingface.co/blog/inference-pro).

You will need a Hugging Face Hub Token to use the ``Nous-Hermes-2-Mixtral-8x7B-DPO`` model from the Inference API. You can get one by signing up on the [Hugging Face Hub](https://huggingface.co/join).

If you prefer using a local model, you can provide Action engine with a DefaultLocalLLM, or import the Hugging Face model of your choice. The default local model is ``HuggingFaceH4/zephyr-7b-gemma-v0.1``.

We will use a ``bge-small-en-v1.5`` to perform semantic search, but you can provide the embedder of your choice.

We use a specific prompt template that leverages Few shot learning with Chain of Thought to ensure the model performs correctly for our use case of Selenium code generation.

You can have a look at the template [here](https://github.com/dhuynh95/LaVague/blob/main/prompt_template.txt).

# Set up

In [ ]:
!apt install ca-certificates fonts-liberation unzip \
    libappindicator3-1 libasound2 libatk-bridge2.0-0 libatk1.0-0 libc6 \
    libcairo2 libcups2 libdbus-1-3 libexpat1 libfontconfig1 libgbm1 \
    libgcc1 libglib2.0-0 libgtk-3-0 libnspr4 libnss3 libpango-1.0-0 \
    libpangocairo-1.0-0 libstdc++6 libx11-6 libx11-xcb1 libxcb1 \
    libxcomposite1 libxcursor1 libxdamage1 libxext6 libxfixes3 libxi6 \
    libxrandr2 libxrender1 libxss1 libxtst6 lsb-release wget xdg-utils

!wget https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chrome-linux64.zip
!wget https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chromedriver-linux64.zip
!unzip chrome-linux64.zip
!unzip chromedriver-linux64.zip
!rm chrome-linux64.zip chromedriver-linux64.zip

In [ ]:
!pip install lavague
!pip install gradio

## Code execution in action

In [3]:
from llama_index.llms.huggingface import HuggingFaceInferenceAPI
# If you want to use Hugging Face inference api, set up your Hugging Face api token here
import os

try:
  from google.colab import userdata
  HF_TOKEN = userdata.get('HF_TOKEN')
except:
  import os
  HF_TOKEN = os.environ["HF_TOKEN"]

if not HF_TOKEN:
  from getpass import getpass
  HF_TOKEN = getpass('Enter your HF Token (https://huggingface.co/docs/hub/en/security-tokens): ')

model_id = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
max_new_tokens = 512

# Monkey patch because stream_complete is not implemented in the current version of llama_index
from llama_index.core.base.llms.types import (
    CompletionResponse,
)
def stream_complete(
    self, prompt: str, formatted: bool = False, **kwargs
):
  def gen():
    text = ""
    for x in self._sync_client.text_generation(
            prompt, **{**{"max_new_tokens": self.num_output, "stream": True}, **kwargs}
        ):
      text += x
      yield CompletionResponse(text=text, delta=x)
  return gen()

HuggingFaceInferenceAPI.stream_complete = stream_complete

llm = HuggingFaceInferenceAPI(model_name=model_id, token=HF_TOKEN, num_output=max_new_tokens)

from lavague.default_prompt import DEFAULT_PROMPT
from lavague.ActionEngine import extract_first_python_code

prompt_template = DEFAULT_PROMPT
cleaning_function = extract_first_python_code

/home/daniel/miniconda3/envs/code_rewrite/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Beginning

In [1]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [1]:
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "HuggingFaceH4/zephyr-7b-gemma-v0.1"

quantization_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)

# We will stop generation as soon as the model outputs the end of Markdown to make inference faster
stop_token_id = [tokenizer.convert_tokens_to_ids("```"), tokenizer.convert_tokens_to_ids("``")]
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer, max_new_tokens=1024, stopping_ids=stop_token_id)

from lavague.default_prompt import GEMMA_PROMPT
from lavague.ActionEngine import ActionEngine

prompt_template = DEFAULT_PROMPT
cleaning_function = lambda code: code.split("```")[0]

/home/daniel/miniconda3/envs/code_rewrite/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:  75%|███████▌  | 3/4 [00:58<00:19, 19.61s/it]

In [4]:
import gradio as gr
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

from lavague.defaults import DefaultEmbedder
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

MAX_CHARS = 1500

# Use this action_engine instead to have a local inference
# action_engine = ActionEngine(llm=DefaultLocalLLM())

action_engine = ActionEngine(llm=llm, embedding=DefaultEmbedder(), prompt_template=prompt_template, cleaning_function=cleaning_function)

## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1600,900")

# Set path to chrome/chromedriver as per your configuration

try:
    import google.colab
    chrome_options.binary_location = "/content/chrome-linux64/chrome"
    webdriver_service = Service("/content/chromedriver-linux64/chromedriver")
except:
    import os.path
    homedir = os.path.expanduser("~")
    chrome_options.binary_location = f"{homedir}/chrome-linux64/chrome"
    webdriver_service = Service(f"{homedir}/chromedriver-linux64/chromedriver")


title = """
<div align="center">
  <h1>🌊 Welcome to LaVague</h1>
  <p>Redefining internet surfing by transforming natural language instructions into seamless browser interactions.</p>
</div>
"""

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

# action_engine = ActionEngine(llm, embedder)

def process_url(url):
    driver.get(url)
    driver.save_screenshot("screenshot.png")
    # This function is supposed to fetch and return the image from the URL.
    # Placeholder function: replace with actual image fetching logic.
    return "screenshot.png"

def process_instruction(query, url_input):
    if url_input != driver.current_url:
        driver.get(url_input)
    html = driver.page_source
    code, source_nodes = action_engine.get_action(query, html)
    print(code)
    return code, source_nodes

def exec_code(code, source_nodes, full_code):
    html = driver.page_source
    try:
        exec(code)
        output = "Successful code execution"
        status = """<p style="color: green; font-size: 20px; font-weight: bold;">Success!</p>"""
        full_code += code
    except Exception as e:
        output = f"Error in code execution: {str(e)}"
        status = """<p style="color: red; font-size: 20px; font-weight: bold;">Failure! Open the Debug tab for more information</p>"""
    return output, code, html, status, full_code

def update_image_display(img):
    driver.save_screenshot("screenshot.png")
    url = driver.current_url
    return "screenshot.png", url

def show_processing_message():
    return "Processing..."

def update_image_display(img):
    driver.save_screenshot("screenshot.png")
    url = driver.current_url
    return "screenshot.png", url

def create_demo(base_url, instructions):
  with gr.Blocks() as demo:
      with gr.Tab("LaVague"):
        with gr.Row():
            gr.HTML(title)
        with gr.Row():
            url_input = gr.Textbox(value=base_url, label="Enter URL and press 'Enter' to load the page.")
        
        with gr.Row():
            with gr.Column(scale=7):
                image_display = gr.Image(label="Browser", interactive=False)
            
            with gr.Column(scale=3):
                with gr.Accordion(label="Full code", open=False):
                    full_code = gr.Code(value="", language="python", interactive=False)
                code_display = gr.Code(label="Generated code", language="python",
                                        lines=5, interactive=True)
                
                status_html = gr.HTML()
        with gr.Row():
            with gr.Column(scale=8):
                text_area = gr.Textbox(label="Enter instructions and press 'Enter' to generate code.")
                gr.Examples(examples=instructions, inputs=text_area)
      with gr.Tab("Debug"):
        with gr.Row():
            with gr.Column():
                log_display = gr.Textbox(interactive=False, lines=20)
            with gr.Column():
                source_display = gr.Code(language="html", label="Retrieved nodes", interactive=False, lines=20)
        with gr.Row():
            with gr.Accordion(label="Full HTML", open=False):
                full_html = gr.Code(language="html", label="Full HTML", interactive=False, lines=20)
  
      # Linking components
      url_input.submit(process_url, inputs=url_input, outputs=image_display)
      text_area.submit(show_processing_message, outputs=[status_html]).then(
          process_instruction, inputs=[text_area, url_input], outputs=[code_display, source_display]
          ).then(
          exec_code, inputs=[code_display, source_display, full_code], 
          outputs=[log_display, code_display, full_html, status_html, full_code]
      ).then(
          update_image_display, inputs=image_display, outputs=[image_display, url_input]
      )
  demo.launch(share=True, debug=True)

You can now try the demo with either the Hugging Face or the irs website!

In [5]:
base_url = "https://huggingface.co/"

instructions = ["Click on the Datasets item on the menu, between Models and Spaces",
                "Click on the search bar 'Filter by name', type 'The Stack', and press 'Enter'",
                "Scroll by 500 pixels",]

create_demo(base_url, instructions)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://dd730b6846da0a7662.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/daniel/miniconda3/envs/code_rewrite/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


# First we need to identify the menu item first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the menu item.
# The menu item can be identified using the text "Datasets"
datasets_menu_item = driver.find_element(By.XPATH, "//a[contains(text(), 'Datasets')]")

# Then we can click on it
datasets_menu_item.click()



/home/daniel/miniconda3/envs/code_rewrite/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


# First we need to identify the component first, then we can click on it.

# Based on the HTML, the link can be uniquely identified using the class "w-full rounded-full border border-gray-200 text-sm placeholder-gray-400 shadow-inner outline-none focus:shadow-xl focus:ring-1 focus:ring-inset dark:bg-gray-950 h-7 pl-7"
# Let's use this class with Selenium to identify the link
search_bar = driver.find_element(By.XPATH, "//*[@class='w-full rounded-full border border-gray-200 text-sm placeholder-gray-400 shadow-inner outline-none focus:shadow-xl focus:ring-1 focus:ring-inset dark:bg-gray-950 h-7 pl-7']")

search_bar.click()

# Now we can type the asked input
search_bar.send_keys("The Stack")

# Finally we can press the 'Enter' key
search_bar.send_keys(Keys.ENTER)



/home/daniel/miniconda3/envs/code_rewrite/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


# We don't need to use the HTML data as this is a stateless operation.
# 500 pixels should be sufficient. Let's execute the JavaScript to scroll up.

driver.execute_script("window.scrollBy(0, 500)")



In [ ]:
base_url = "https://www.irs.gov/"

instructions = ["Click on the 'Pay' item on the menu, between 'File' and 'Refunds'",
                "Click on 'Pay Now with Direct Pay' just below 'Pay from your Bank Account'",
                "Click on 'Make a Payment', just above 'Answers to common questions'",]

create_demo(base_url, instructions)